In [1]:
from time import sleep
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
hym = pd.read_csv("../hymnary.csv", index_col="id")

In [39]:
def scrape_info(title):
    r = requests.get(f"https://www.hymnal.net/zh/search/h/all/{title}")
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    # Get the first link item in the list-group div
    link_item = soup.select_one(".list-group a")

    # Extract the text and link number from the link item
    text = link_item.contents[0].strip() if link_item else ""
    link = 'https://www.hymnal.net' + link_item["href"] if link_item else None

    sleep(0.1)
    return {"match_title_zh": text, "match_url": link}

In [40]:
scrape_info('讓我愛而不受感戴')

{'match_title_zh': '讓我愛', 'match_url': 'https://www.hymnal.net/zh/hymn/ts/428'}

In [41]:
best_match = hym.query('true_match != True')['name_ch'].apply(scrape_info)

In [43]:
matches = hym.drop(columns='match_url').query('true_match != True').reset_index().join(pd.json_normalize(best_match))
matches

,id,name_ch,name,match_title,match_id,similarity,true_match,match_title_zh,match_url
0,1,榮耀歸與最高神,Glory To God In The Highest,When I've run the race before me,8773.0,0.000,NaN,,None
1,3,三一頌,Doxology,NaN,NaN,0.000,NaN,,None
2,4,當讚美聖父,Praise Ye The Father,"O God and Father, we our praises bring",54.0,0.000,NaN,,None
3,6,主阿你的名何其美,How Excellent Is Thy Name,"O Lord, our Lord, how excellent",1097.0,0.125,NaN,,None
4,7,偉大的神,How Great Thou Art,"My Father God, when on Thy vast creation",17.0,0.000,NaN,,None
...,...,...,...,...,...,...,...,...,...
503,712,又是一年正起首,Another Year Is Dawning,NaN,NaN,0.000,NaN,,None
504,713,感謝神,Thanks To God,Pray to touch the throne of God,783.0,0.000,NaN,新耶路撒冷是神恩典的展示,https://www.hymnal.net/zh/hymn/ts/928
505,714,讓我們重起首,"Come, Let Us Anew","Your ministry, O Lord",1187.0,0.000,NaN,,None
506,715,數主恩惠,Count Your Blessings,When upon life's billows you are tempest tossed,707.0,0.000,NaN,,None


In [45]:
matches[['id', 'name_ch', 'match_title_zh', 'match_url']].to_csv('matches.csv', index=False)